# CS 39AA - Notebook 3: Improving our Airline Tweet Model

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/CS39AA/blob/main/nb3_improving_our_twitter_model.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/CS39AA/blob/main/nb3_improving_our_twitter_model.ipynb)

We'll now revisit the Airline Tweet dataset and try to resolve some of the issues that identified at the end of our first notebook, [Notebook 1](https://github/sgeinitz/CS39AA/blob/main/nb1_text_data_and_pandas.ipynb).

If you recall, in Notebook 1, we created a very simple ad hoc model that simply counted the most frequently occuring words in each of the three distinct classes (positive, negative, and neutral). Our '_model_' then would take a new tweet and check to see how many times these words appeared. If more positive words appeared, then we assigned that label, and so on for negative and neutral. 

At the end we listed three issues or areas for improvement. These were:
1. __Better Tokenization:__ The tokenization is very crude right now. Simply splitting a tweet into tokens with the 'SPACE' character as a delimiter means that positive tokens like _"awesome."_ and _"awesome!"_ are considered as two different tokens. Cases such as that one are relatively easy to solve since it just meaning removing some punctuation. However, different forms of a word, such as _"delay"_ and _"delayed"_ can cause some issues too. For our ad hoc modeling approach above, resolving these tokenization don't seem to affect the outcome too much right now. But, once we try to use a proper model (e.g. naive bayes, neural network, etc.), then these tokenization deficiences can affect model performance even more. For example, having all of the different forms of the verb _"delay"_ will mean that our model needs to have that many more parameters in it. 
2. __Vectorization__: Even with better tokenization, we still need to do more modify the data to be able to use other types of models. What we specifically need to do is convert the tokens into a numerical representation of some kind. When working text data, this process of converting text/tokens into a numerical representation will allow us to use many different types of models, including neural networks. 
3. __Modeling Process/Evaluation:__ The modeling process and assessment need to be improved. To start, our accuracy of {{mod_accuracy}} is not much better than if we simply label every tweet as negative (since 65% of all tweets are negative). So we need to be sure that we're comparing our model performance metrics to a suitable baseline. The larger issue, however, is that we don't have a validation and/or test set right now. We used all 10k observations to build our positive, negative, and neutral sets of words; then we check our accuracy on these exact same sets of tweets. To really understand how our model will work for a new tweet that we have never seen, and that is posted in the future, we need to remove a portion of the dataset from the 'training', then evaluate our model against this. 

### 1. Our ad hoc Model Improved
Let's reexamine our ad hoc model with all of those issues/improvements addressed. As before, we'll open the data file that is hosted on github and quickly check it's shape and peek at the first few rows (after changing the default display width for a pandas dataframe column). 

In [ ]:
import pandas as pd

In [ ]:
data_URL = 'https://raw.githubusercontent.com/sgeinitz/CS39AA/main/data/trainA.csv'
df = pd.read_csv(data_URL)
print(f"df.shape: {df.shape}")
pd.set_option("display.max_colwidth", 240)
df.head(10)

Recall that about 2/3 of the data have negative labels, and that the remaining labels are roughly split between positive and neutral (slightly more neutral than positive). 

In [ ]:
df.sentiment.value_counts(normalize=True)

Let's start with the nltk TweetTokenizer, which will split the text into separate words and characters based on common Twitter conventions. 

In [ ]:
from nltk.tokenize import TweetTokenizer
tk = TweetTokenizer()
df['tokens_raw'] = df['text'].apply(lambda x: tk.tokenize(x.lower()))
df.head()

Next, let's remove common stop words (e.g. "_the_", "_in_", etc.). In this next cell we will also remove some characters/punctuation, as well as hashtag tokens. 

In [ ]:
import re
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
chars2remove = set(['.','!','/', '?'])
df['tokens_raw'] = df['tokens_raw'].apply(lambda x: [w for w in x if w not in stops])
df['tokens_raw'] = df['tokens_raw'].apply(lambda x: [w for w in x if w not in chars2remove])
df['tokens_raw'] = df['tokens_raw'].apply(lambda x: [w for w in x if not re.match('^#', w)]) # remove hashtags
#df['tokens_raw'] = df['tokens_raw'].apply(lambda x: [w for w in x if not re.match('^http', w)]) # remove web links
#df['tokens_raw'] = df['tokens_raw'].apply(lambda x: [w for w in x if not re.match('^@', w)]) # remove web links

df.head()

For the final step of text pre-processing we will lemmatize the tokens. Note that there are much better ways to do this but that we want to use a simple lemmatizer. For example, some lemmatizers also utilize a model internally to predict the part-of-speech for each word, since whether the word is a noun, adjective, verb, etc. will affect how lemmatization is done. Since we want to keep things simple here, and focus only on the lemmatization step, we'll assume every word is the same part of speech. Note that this is not by any means ideal (try to identify the incorrectly lemmatized token in the five tweets printed out below). In practice we would certainly utilize a 'smarter' lemmatizer .

In [ ]:
from nltk.stem import WordNetLemmatizer
# also need to run following one time on your system (can be done outside of this notebook)
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens_raw'].apply(lambda x: [lemmatizer.lemmatize(w, pos="v") for w in x])
#df['tokens'] = df['tokens_raw'].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
df.head()

Since each tweet is currently stored as a string we then created a new column that was a list of each of the words in the tweet (since the default delimiter is a space character). 

In [ ]:
# before we do the prediction let's split the dataset into validation and training subsets
# (note that we could use sklearn.model_selection.train_test_split() for this, but we'll do it manually here)
import random
random.seed(42)

df_full = df.copy()
assert df_full.shape[0] == 10000

indices = list(range(df_full.shape[0]))

random.shuffle(indices)

train_indices = indices[:8500]
test_indices = indices[8500:]

# df will be the in-sample training dataset
df = df_full.iloc[train_indices,:].copy()
print(f"df.shape: {df.shape}")

# df_test will be the out-of-sample validation dataset
df_test = df_full.iloc[test_indices,:].copy()
print(f"df_test.shape: {df_test.shape}")
df_test.head()

Next, we created a vocabularly sorted by frequency for the full dataset, the subset of positive tweets, negative tweets, and neutral tweets. 

In [ ]:
# subset the data by positive, negative, and neutral
df_pos = df[df['sentiment'] == 'positive']
df_neg = df[df['sentiment'] == 'negative']
df_neu = df[df['sentiment'] == 'neutral']

def create_vocab_list(tokens_column):
    vocab = dict()
    for tweet_tokens in tokens_column:
        for token in tweet_tokens:
            if token not in vocab:
                vocab[token] = 1
            else:
                vocab[token] += 1
    return vocab

vocab_all = dict(sorted(create_vocab_list(df['tokens']).items(), key=lambda item: item[1], reverse=True))
vocab_pos = dict(sorted(create_vocab_list(df_pos['tokens']).items(), key=lambda item: item[1], reverse=True))
vocab_neg = dict(sorted(create_vocab_list(df_neg['tokens']).items(), key=lambda item: item[1], reverse=True))
vocab_neu = dict(sorted(create_vocab_list(df_neu['tokens']).items(), key=lambda item: item[1], reverse=True))
print(f"number of unique tokens overall: {len(vocab_all)}, pos tokens: {len(vocab_pos)}, neg: {len(vocab_neg)}, neu: {len(vocab_neu)}")

Recall that in our original notebook we had about ~20k unique tokens, and this is now down to ~10k. Even though this is an ad hoc model, it is likely still a good simplification of the data. If we were using a real model, then this will be really beneficial since it will directly translate to the number of parameters in our model (decreasing the number of parameters is usually good, particularly when we can maintain similar performance, and/or we have a small dataset). As we did last time, let's look at the most frequently occuring words across all classes (ignoring whether it's positive, negative, or neutral).

In [ ]:
list(vocab_all.items())[:20]

Even though we already removed stop words during our tokenization step, it still seems from the output above that many of the most common occuring words across the full dataset are not very informative. So we will likely still end up benefiting from removing some of these words from the positive, negative, and neutral vocabularies that we created. The value of $500$ that we chose seems arbtrary here but in practice we would assess the model performance (on the validation dataset) for many different values both less than and greater than $500$. Even though our model is ad hoc, we can this of this number as a [__hyperparameter__](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)) since it is not directly learned from the data but can be used to tune the performance of the model. In practice, we would want to look at the accuracy for both the training and validation data across many different values. 

In [ ]:
top_n_to_remove = 1000 #500 #500 #1000 # for our ad hoc model we can think of this as a type of hyperparameter
for i, item in enumerate(vocab_all.items()):
    if i == top_n_to_remove:
        break
    #print(f" removing token: {item[0]:15} (w/ freq = {item[1]:5}) from vocabs")
    if item[0] in vocab_pos:
        del vocab_pos[item[0]]
    if item[0] in vocab_neg:
        del vocab_neg[item[0]]
    if item[0] in vocab_neu:
        del vocab_neu[item[0]]

print(f"number of unique tokens overall: {len(vocab_all)}, pos tokens: {len(vocab_pos)}, neg: {len(vocab_neg)}, neu: {len(vocab_neu)}")

In [ ]:
list(vocab_pos.items())[:10]

In [ ]:
list(vocab_neg.items())[:10]

In [ ]:
list(vocab_neu.items())[:10]

As before, those words look pretty good since they seem to have a positive, negative, neutral connotation, respectively.  Now, let's try classifying the tweets by looking at one and counting how many tokens it has from the top k tokens in the vocab_pos, vocab_neg, and vocab_neutral sets. Whichever vocab it has the greatest number of tokens from, let's classify it as that. 

To accomplish this let's first create a single object here that represents our 'model'. This object is a dictionary data type and holds the vocabulary for each of positive, negative, and neutral classes. Note that below we are including _all_ of the tokens for each label because we are simply using a '[:]' in the square brackets, but we could easily include just the top $p$ positive tokens, top $p$ negative, etc. for $p$ equal to, say, $100$ using '[:100]'. This can be considered another hyperparameter for ad hoc model. 

In [ ]:
# put positive, negative, neutral words together into a single object to create our 'model'
classifier_tokens = {"positive": list(vocab_pos.keys())[:], "negative": list(vocab_neg.keys())[:], "neutral": list(vocab_neu.keys())[:]}

# a sample tweet to classify
tweet2classify_i = 45
tweet2classify = df.iloc[tweet2classify_i,:]['tokens']
print(f"example of tweet to classify: \n  sentiment = {df.iloc[tweet2classify_i,0]} \n  tweet = {df.iloc[tweet2classify_i,1]}")

pos = 0
neg = 0
neu = 0
for tok in tweet2classify:
    if tok in classifier_tokens['positive']:
        pos += 1
    elif tok in classifier_tokens['negative']:
        neg += 1
    elif tok in classifier_tokens['neutral']:
        neu += 1

print(f"\n vocab counts: pos = {pos},  neg = {neg},  neu = {neu}")

Let's define a function to predict the sentiment of a tweet using our ad hoc approach of counting the occurrence of words from the positive, negative, and neutral vocabularies. 

In [ ]:
def predict_tweet_sentiment(tweet_tokens):
    pos = 0
    neg = 0
    neu = 0
    for tok in tweet_tokens:
        if tok in classifier_tokens['positive']:
            pos += 1
        elif tok in classifier_tokens['negative']:
            neg += 1
        elif tok in classifier_tokens['neutral']:
            neu += 1
    if pos > neg and pos > neu:
        return "positive"
    elif neu > pos and neu > neg:
        return "neutral"
    else:
        return "negative"

In [ ]:
# split dataset into train and validation/test


Make predictions for the training dataset and check the performance with accuracy and a confusion matrix. 

In [ ]:
# make predictions for training dataset
df['predicted_sentiment'] = df['tokens'].apply(lambda x: predict_tweet_sentiment(x))

# check performance on training dataset
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
disp = ConfusionMatrixDisplay(confusion_matrix(df['sentiment'], df['predicted_sentiment']), display_labels=['negative', 'neutral', 'positive'])
disp.plot()
mod_accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
print(f"our ad hoc model's accuracy on training dataset: {mod_accuracy*100:.2f}%")

Let's now assess the performance of our model on the validation data (i.e. 'df_test'). 

In [ ]:
# make predictions for the validation/test dataset
df_test['predicted_sentiment'] = df_test['tokens'].apply(lambda x: predict_tweet_sentiment(x))

# check performance on validation/test dataset
disp = ConfusionMatrixDisplay(confusion_matrix(df_test['sentiment'], df_test['predicted_sentiment']), display_labels=['negative', 'neutral', 'positive'])
disp.plot()
mod_accuracy = accuracy_score(df_test['sentiment'], df_test['predicted_sentiment'])
print(f"our ad hoc model's accuracy on validation dataset: {mod_accuracy*100:.2f}%")

Let's stop here and think about whether we are assessing the model correctly. That is:
1. Are we using a training and validation datasets to get an idea of how our model's  __out-of-sample__ performance? 
2. Also, are we splitting the dataset at the right time to ensure there is no [__data leakage__](https://en.wikipedia.org/wiki/Leakage_(machine_learning))? 

### 2. Using a "real" ML Model

Now that we have tokenized the data, we can quickly vectorize it to get a numeric representation for each tweet. That will then give us the flexibility to select from a much broader range of commonly used machine learning models. We can still use the `df` data frame as the training dataset, and `df_test` as the validation dataset. 

Just to be sure, let's go ahead and drop the `predicted_sentiment` column from both data frames. We can also drop the `tokens_raw` column since we are no longer using it. 

In [ ]:
if 'predicted_sentiment' in df.columns:
    print("removing column, predicted_sentiment, from df")
    df.drop('predicted_sentiment', axis=1, inplace=True)

if 'predicted_sentiment' in df_test.columns:
    print("removing column, predicted_sentiment, from df_test")
    df_test.drop('predicted_sentiment', axis=1, inplace=True)

print(f"df.shape: {df.shape}")
print(f"df_test.shape: {df_test.shape}")
df.head()

The input to the sklearn vectorizer function requires that each observation (i.e. tweet) is in the form of a string, rather than a list of tokens. So we first need to combine the individual tokens for each tweet back into a string, which we do here:

In [ ]:
df['textclean'] = df['tokens'].apply(lambda x: ' '.join(x))
df.head()

Now we will load the term-frequency inverse-document-frequency vectorizer from sklearn, `TfidfVectorizer`, to convert each tweet into a vector. We'll go ahead and call the resulting vectorized data, `X`, or `X_train` since it is only the training dataset. As with conventional statistical models, "_X_" represents the set of predictors, or independent variables. 

Also, note that `TfidfVectorizer` is a powerful text processing object. It has the ability to remove stop words, strip symbols, and do much of the work that our manual tokenization did. As such, we could easily use the original tweet text here, but we'll go ahead and continue with our manually tokenized data in the column, `textclean`. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#import seaborn as sns
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(df['textclean']).toarray()
#X_train = tfidf_vectorizer.fit_transform(df['text']).toarray() # original tweet text (without our manual tokenization)

#from sklearn.feature_extraction.text import CountVectorizer
#one_hot_vectorizer = CountVectorizer(binary=True)
#tweet_matrix = one_hot_vectorizer.fit_transform(list(df['textclean']))#.toarray()

print(f"X_train.shape = {X_train.shape}")
type(X_train)

Note that the data type for `X_train` is no longer a pandas data frame. Instead, it is a numpy ndarray. We will talk much more about the python module, numpy, and its data types in the coming week. For now, let's just think of it a more efficently implemented object than a pandas data frame (e.g. smaller memory footprint), and one that we can think of a more of a matrix than a table of data. This can be seen by how we can easily index into `X_train` without the need for methods such as `iloc` or `loc`. Here we will look at the first 5 rows and 10 columns.

In [ ]:
X_train[:5, :10]

Looking at the output above, and back at the dimensions of `X_train` two cells back, what does this tell you about the data (aside from the size of the training data)? Specifically, what do the number of columns represent?

If you're not sure, then look at the output of the following cell. We are looking at the first row of the training dataset, `X_train` again, but looking only at the non-zero values. These are the term-frequency inverse-document-frequency values for this tweet. 

In [ ]:
obs1 = list(X_train[0,:])
for i, tfidf_val in enumerate(obs1):
    if tfidf_val > 0:
        print(f"obs1 token at column i={i}, has a non-zero TF-IDF value: {tfidf_val}")

If we wanted to, we could retrieve the token associated with each column of `X_train` by using the `tfidf_vectorizer`. Let's do that now just to see if we can try to recover the tweet. You'll notice that we can't recover the ordering of the words, but rather just the set of words that were in the tweet. 

In [ ]:
obs1 = list(X_train[0,:])
for i, tfidf_val in enumerate(obs1):
    if tfidf_val > 0:
        print(f"obs1 has a non-zero TF-IDF value: {tfidf_val} at col i={i} (associated with token: {tfidf_vectorizer.get_feature_names_out()[i]})")

We can verify this by going back to our training dataset, `df`, and checking what the first observation was. Looking at the output below. As stated above, we see that the same words are present in the data but not necessarily in the same order. Disregarding word order when vectorizing text data is sometimes (informally) referred to as a "[__bag-of-words__](https://en.wikipedia.org/wiki/Bag-of-words_model)" approach. Often times, "__bag-of-words__" suggests that just the counts (i.e. __term frequency__) is used, rather than what we are using here, __term-frequency inverse-document-frequency__. Nonetheless, the underlying idea that words frequency is accounted for, but word order is neglected, is an important observation to make here.

In [ ]:
df.iloc[0,:]

A good default model to start with that does not require too much fine tuning, and generally performs well is a [__Random Forest__](https://en.wikipedia.org/wiki/Random_forest) model. We won't dig into the details of how an RF model works exactly, but so long as we know that it creates many decision trees on random subsets of the data, then you've got a decent idea. Before doing so, we also need to convert the labels (i.e. `df.sentiment`) to a numpy data type. We'll quickly do that now and look at the first few observations. 

In [ ]:
y_train = df.sentiment.to_numpy()
y_train[:10]

We'll not fit the random forest model on the training data and check the performance. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model = model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
predictions_train

disp = ConfusionMatrixDisplay(confusion_matrix(df['sentiment'], predictions_train), display_labels=['negative', 'neutral', 'positive'])
disp.plot()
print(f"accuracy (on X_train): {accuracy_score(df['sentiment'], predictions_train):.4f}")

Wow! Accuracy of ~99% is much better than what we saw with our ad hoc model. Let's check and see how well the model does on the validation dataset now.

Notice that we are using `tfidf_vectorizer.transform()` now, and not the same method we used before on the training dataset, which was `tfidf_vectorizer.fit_transform()`. The `fit_transform` method creates the vocabulary from the training data. The `transform` method will use the vocabulary that was previously made, but if it encounters a token in the validation data that was never seen in the training data, then it will simply ignore it. This is one reason why making sure that training data for models is regularly updated.

In [ ]:
df_test['textclean'] = df_test['tokens'].apply(lambda x: ' '.join(x))
X_test = tfidf_vectorizer.transform(df_test['textclean']).toarray()
X_test.shape

In [ ]:
predictions_test = model.predict(X_test)
disp = ConfusionMatrixDisplay(confusion_matrix(df_test['sentiment'], predictions_test), display_labels=['negative', 'neutral', 'positive'])
disp.plot()
print(f"accuracy (on X_test): {accuracy_score(df_test['sentiment'], predictions_test):.4f}")

Okay, the model performance on the validation data is not nearly as good as the performance on the training dataset. That's expected to some degree. 

We may be able to improve it slightly by finding the optimal value of one of the random forest hyperparameters, e.g. `n_estimators`. However, one of the strengths of random forest is not it does a good job of overfitting. Knowing that, how do you think smaller values of `n_estimators` will affect accuracy on the validation data? How do you think larger values will affect it? Try a few different values of `n_estimators`, both larger and smaller than the default of `n_estimators=100`, to see what happens.

Below we go ahead and try many different hyperparameter values (for n_estimators) to see what the best value is. As can be seen, there is not an immediately obvious choice for the best value of the n_estimators hyperparameter, which is exactly why random forest is such a nice to model to use at the beginning. Admittedly, an RF model might not perform as well as some other machine learning models, but it is reliable and does a decent job at balancing the Bias-Variance Tradeoff right away without a lot of tuning. 

In [ ]:
from sklearn.metrics import log_loss

# A function to create and fit a RF with a specific number of trees
def fitRFModel(hyperparam_value):
    rf_model = RandomForestClassifier(n_estimators=hyperparam_value, random_state=5)
    #rf_model = RandomForestClassifier(min_samples_split=hyperparam_value, random_state=1)
    rf_model.fit(X_train, y_train)
    y_train_pred_prob = rf_model.predict_proba(X_train)
    y_train_pred = rf_model.predict(X_train)
    y_val_pred_prob = rf_model.predict_proba(X_test)
    y_val_pred = rf_model.predict(X_test)
    train_loss = log_loss(df['sentiment'], y_train_pred_prob, labels=['negative', 'neutral', 'positive']) #, F.binary_cross_entropy(torch.tensor(y_train_pred), torch.tensor(y_train.to_numpy().astype(float)), reduction="mean")
    train_acc = accuracy_score(df['sentiment'], y_train_pred)
    val_loss = log_loss(df_test['sentiment'], y_val_pred_prob, labels=['negative', 'neutral', 'positive']) #F.binary_cross_entropy(torch.tensor(y_val_pred), torch.tensor(y_val.to_numpy().astype(float)), reduction="mean")
    val_acc = accuracy_score(df_test['sentiment'], y_val_pred)
    #return((train_loss.item(), val_loss.item()))
    return (train_loss, val_loss, train_acc, val_acc)

# Possible values of min_samples_split are 10 to 70 (by 5)
hyp_param_vals = list(range(5,151,10))
#hyp_param_vals = [5] + list(range(10,201,10))
metrics = []
for hp in hyp_param_vals:
    metrics.append(fitRFModel(hp))

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 6))
ax = fig.add_axes([0, 0, 1, 1]) #.1, 0.1, 0.8, 0.8]) # main axes
ax.plot(hyp_param_vals, [metric[1] for metric in metrics], '--ro') # validattion loss
ax.plot(hyp_param_vals, [metric[0] for metric in metrics], '--bo') # training loss
ax.legend(["Validation Loss", "Train Loss"], loc=1)
ax.set_xticks(hyp_param_vals)
ax.set(xlabel="n_estimators", ylabel="loss (lower is better)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_axes([0, 0, 1, 1]) #.1, 0.1, 0.8, 0.8]) # main axes
ax.plot(hyp_param_vals, [metric[3] for metric in metrics], '--ro') # validattion accuracy
ax.plot(hyp_param_vals, [metric[2] for metric in metrics], '--bo') # training accuracy
ax.legend(["Validation Accuracy", "Train Accuracy"], loc=4)
ax.set_xticks(hyp_param_vals)
ax.set(xlabel="n_estimators", ylabel="accuracy (higher is better)")
plt.show()